Now we want to populate our data repository

In [1]:
import json
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import simplesimdb as simplesim # this one is our own database manager
import subprocess # to capture errors from the simulations
import itertools

In [2]:
m = simplesim.Manager(directory='test', executable='./execute.sh', filetype='nc')

In [3]:
init2x= {"shear":[0.0,6.283185307179586],
         "mms": [-1.0,1.0]
         }
init2bc= {"shear":["PER", "DIR"],
          "mms": ["DIR", "PER"]}

init2init =  {
    "shear": {"type": "shear", "rho": 0.20943951023931953, "delta": 0.2 },
    "mms":   {"type": "mms", "sigma": 0.2, "velocity": 1.0}
}

n2v={
     "shear": {
    1:[144,1e-2],
    3:[48, 5e-3],
    4:[36, 2.5e-3],
    5:[28, 2e-3],
    7:[20, 1e-3]
    },
    "mms": {
    1:[144,2e-2],
    3:[48, 2e-3],
    4:[36, 2e-3],
    5:[28, 1e-3],
    7:[20, 5e-4]
    }
}

endtime = {"shear": 10, "mms": 1}

maxout = {"shear": 40, "mms": 5}
reg2reg = {"none" : {"type" : "none"},
           "modal" : {"type" : "modal", "alpha" : 36, "order" : 8, "eta_c" : 0.5},
          "viscous": {"type" : "viscosity", "order" : 1,
                      "nu" : 1e-3, "direction" : "forward"},
          "hyperviscous" : {"type": "viscosity", "order" : 2,
                            "nu" : 5e-4, "direction" : "forward"}
          }

In [4]:
def generate_standard_input( n, init, multiplication, advection, regularization) :
    itstp=endtime[init]/maxout[init]/n2v[init][n][1]
    if not itstp.is_integer() :
        print("Warning: itstp is not an integer!")
    inputfile={
        "grid":
        {   
            "type": "Cartesian2d",
            "n"  : n,
            "Nx" : n2v[init][n][0],
            "Ny" : n2v[init][n][0],
            "x": init2x[init],
            "y": init2x[init],
            "bc" : init2bc[init]
        },
        "timestepper":
        {   
            "type": "FilteredExplicitMultistep",
            "tableau": "eBDF-3-3",
            "dt" : n2v[init][n][1]
        },
        "regularization":  reg2reg[regularization],
        "output":
        {   
            "type": "netcdf",
            "itstp"   : int(itstp),
            "maxout"  : int(maxout[init])
        },
        "advection":
        {   
            "multiplication": multiplication,
            "type": advection
        },
        "elliptic":
        {   
            "type" : "multigrid",
            "stages": 3,
            "eps_pol" : [1e-6,10.0,10.0],
            "direction" : "forward"
        },
        "init": init2init[init]
    }
    if regularization == "viscous" or regularization == "hyperviscous" :
        inputfile["timestepper"]["type"] = "ImExMultistep"
        inputfile["timestepper"]["tableau"] = "ImEx-BDF-3-3"
        inputfile["timestepper"]["eps_time"] = 1e-9
    return inputfile

In [5]:
###### THE RED BUTTON
#m.delete_all()

In [6]:
v_n=[1,3,4,5]
v_init=["shear", "mms"]
v_multiplication=["pointwise", "projection"]
v_advection=["arakawa","centered", "upwind"]
v_regularization=["none", "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])   
    # these crashed on a first attempt
    if(( t == (3, 'shear', 'pointwise', 'centered', 'none')) 
        or (t==(3, 'shear', 'projection', 'arakawa', 'none'))
        or (t==(4, 'shear', 'pointwise', 'centered', 'none'))
        or (t==(4, 'shear', 'projection', 'arakawa', 'none'))
        or (t==(4, 'shear', 'projection', 'centered', 'none'))
        or (t==(5, 'shear', 'pointwise', 'arakawa', 'none'))
        or (t==(5, 'shear', 'projection', 'arakawa', 'none'))
           ):
        print(t, " Crashes!! ")
        continue
    
#     #Check for existing data
#     if not m.exists(inputfile):
#         print(t)
#     uncomment the following to create data
    try:
        print(t)
        m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)
        break

(1, 'shear', 'pointwise', 'arakawa', 'none')
(1, 'shear', 'pointwise', 'arakawa', 'modal')
(1, 'shear', 'pointwise', 'centered', 'none')
(1, 'shear', 'pointwise', 'centered', 'modal')
(1, 'shear', 'pointwise', 'upwind', 'none')
(1, 'shear', 'pointwise', 'upwind', 'modal')
(1, 'shear', 'projection', 'arakawa', 'none')
(1, 'shear', 'projection', 'arakawa', 'modal')
(1, 'shear', 'projection', 'centered', 'none')
(1, 'shear', 'projection', 'centered', 'modal')
(1, 'shear', 'projection', 'upwind', 'none')
(1, 'shear', 'projection', 'upwind', 'modal')
(1, 'mms', 'pointwise', 'arakawa', 'none')
(1, 'mms', 'pointwise', 'arakawa', 'modal')
(1, 'mms', 'pointwise', 'centered', 'none')
(1, 'mms', 'pointwise', 'centered', 'modal')
(1, 'mms', 'pointwise', 'upwind', 'none')
(1, 'mms', 'pointwise', 'upwind', 'modal')
(1, 'mms', 'projection', 'arakawa', 'none')
(1, 'mms', 'projection', 'arakawa', 'modal')
(1, 'mms', 'projection', 'centered', 'none')
(1, 'mms', 'projection', 'centered', 'modal')
(1, 'mm

In [7]:
# change the eta_c parameter to 0 for comparison
v_n=[3,4,5]
v_init=["shear"]
v_multiplication=["pointwise","projection"]
v_advection=["arakawa","centered","upwind"]
v_regularization=[ "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4] ) 
    inputfile["regularization"]["eta_c"] = float(0)

    ncfile=''
    try:
        print(t)
        ncfile=m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(3, 'shear', 'pointwise', 'arakawa', 'modal')
(3, 'shear', 'pointwise', 'centered', 'modal')
(3, 'shear', 'pointwise', 'upwind', 'modal')
(3, 'shear', 'projection', 'arakawa', 'modal')
(3, 'shear', 'projection', 'centered', 'modal')
(3, 'shear', 'projection', 'upwind', 'modal')
(4, 'shear', 'pointwise', 'arakawa', 'modal')
(4, 'shear', 'pointwise', 'centered', 'modal')
(4, 'shear', 'pointwise', 'upwind', 'modal')
(4, 'shear', 'projection', 'arakawa', 'modal')
(4, 'shear', 'projection', 'centered', 'modal')
(4, 'shear', 'projection', 'upwind', 'modal')
(5, 'shear', 'pointwise', 'arakawa', 'modal')
(5, 'shear', 'pointwise', 'centered', 'modal')
(5, 'shear', 'pointwise', 'upwind', 'modal')
(5, 'shear', 'projection', 'arakawa', 'modal')
(5, 'shear', 'projection', 'centered', 'modal')
(5, 'shear', 'projection', 'upwind', 'modal')


In [8]:
# Generate one good simulation with double resolution
inputfile = generate_standard_input( 
        n=5, init="shear",multiplication="projection",
        advection="upwind",regularization="none" )
inputfile["grid"]["Nx"] *=2
inputfile["grid"]["Ny"] *=2
inputfile["timestepper"]["dt"]/=2.0
inputfile["output"]["itstp"]*=2

print( inputfile)
ncfile=''
try:
    ncfile=m.create(inputfile)
    #m.delete(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)

{'grid': {'type': 'Cartesian2d', 'n': 5, 'Nx': 56, 'Ny': 56, 'x': [0.0, 6.283185307179586], 'y': [0.0, 6.283185307179586], 'bc': ['PER', 'DIR']}, 'timestepper': {'type': 'FilteredExplicitMultistep', 'tableau': 'eBDF-3-3', 'dt': 0.001}, 'regularization': {'type': 'none'}, 'output': {'type': 'netcdf', 'itstp': 250, 'maxout': 40}, 'advection': {'multiplication': 'projection', 'type': 'upwind'}, 'elliptic': {'type': 'multigrid', 'stages': 3, 'eps_pol': [1e-06, 10.0, 10.0], 'direction': 'forward'}, 'init': {'type': 'shear', 'rho': 0.20943951023931953, 'delta': 0.2}}


In [9]:
# Generate Simulations for first and second order diffusion on Arakawa scheme
v_nu = [1e-4, 2e-4, 5e-4, 1e-3, 2e-3]
v_order = [1,2]
v_n = [1,3,5]
for t in itertools.product( v_n, v_order, v_nu) :
    inputfile = generate_standard_input( 
        n=t[0], init="shear",multiplication="pointwise",
        advection="arakawa",regularization="viscous" )
    inputfile["regularization"]["order"] = t[1]
    inputfile["regularization"]["nu"] = t[2]
    if t[0] == 5 :
        inputfile["timestepper"]["dt"]/=2.0
        inputfile["output"]["itstp"]*=2
    print(t)
    try:
        m.create(inputfile)
    except subprocess.CalledProcessError as e:
        print( e.stderr)


(1, 1, 0.0001)
(1, 1, 0.0002)
(1, 1, 0.0005)
(1, 1, 0.001)
(1, 1, 0.002)
(1, 2, 0.0001)
(1, 2, 0.0002)
(1, 2, 0.0005)
(1, 2, 0.001)
(1, 2, 0.002)
(3, 1, 0.0001)
(3, 1, 0.0002)
(3, 1, 0.0005)
(3, 1, 0.001)
(3, 1, 0.002)
(3, 2, 0.0001)
(3, 2, 0.0002)
(3, 2, 0.0005)
(3, 2, 0.001)
(3, 2, 0.002)
(5, 1, 0.0001)
(5, 1, 0.0002)
(5, 1, 0.0005)
(5, 1, 0.001)
(5, 1, 0.002)
(5, 2, 0.0001)
(5, 2, 0.0002)
(5, 2, 0.0005)
(5, 2, 0.001)
(5, 2, 0.002)


In [10]:
v_n=[1,3,4,5]
v_init=["shear"]
v_multiplication=["pointwise", "projection"]
v_advection=["upwind-advection","centered-advection"]
v_regularization=["none", "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])      

    # Check for existing data
    #if not m.exists(inputfile):
    #    print(t)
    # uncomment the following to create data
    try:
        print(t)
        m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(1, 'shear', 'pointwise', 'upwind-advection', 'none')
(1, 'shear', 'pointwise', 'upwind-advection', 'modal')
(1, 'shear', 'pointwise', 'centered-advection', 'none')
(1, 'shear', 'pointwise', 'centered-advection', 'modal')
(1, 'shear', 'projection', 'upwind-advection', 'none')
(1, 'shear', 'projection', 'upwind-advection', 'modal')
(1, 'shear', 'projection', 'centered-advection', 'none')
(1, 'shear', 'projection', 'centered-advection', 'modal')
(3, 'shear', 'pointwise', 'upwind-advection', 'none')
(3, 'shear', 'pointwise', 'upwind-advection', 'modal')
(3, 'shear', 'pointwise', 'centered-advection', 'none')
(3, 'shear', 'pointwise', 'centered-advection', 'modal')
(3, 'shear', 'projection', 'upwind-advection', 'none')
(3, 'shear', 'projection', 'upwind-advection', 'modal')
(3, 'shear', 'projection', 'centered-advection', 'none')
(3, 'shear', 'projection', 'centered-advection', 'modal')
(4, 'shear', 'pointwise', 'upwind-advection', 'none')
(4, 'shear', 'pointwise', 'upwind-advection', 'mod

In [11]:
# Try to vary timestepper
inputfile = generate_standard_input( 
    n=3, init="shear",multiplication="pointwise",
    advection="upwind-advection",regularization="none")  

inputfile["timestepper"]["tableau"] = "TVB-3-3"
print( "TVB-3-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)
inputfile["timestepper"]["tableau"] = "SSP-6-3"
print( "SSP-6-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr )
 
inputfile["timestepper"]["type"] = "Shu-Osher"
inputfile["timestepper"]["tableau"] = "SSPRK-3-3"
inputfile["timestepper"]["dt"] *= 2
inputfile["output"]["itstp"]/=2
print( "SSPRK-3-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)

TVB-3-3
SSP-6-3
SSPRK-3-3
